## Raster Analytics Workflow

In this example workflow, a user will be shown how to:
* Fetch a raster array
* Return a vsicurl formatted URL for Maxar rasterized objects
* Download a raster image
* Display a raster and manipulate the bands for that raster
* Display a tileset of a raster and manipulate the bands for that raster

### Requirements:
* GDAL (osgeo)
* matplotlib
* numpy
* maxar API token
    * *Note: If you are unfamiliar with maxar API tokens, please refer to the Maxar Geospatial Platform Tokens notebook in this collection*

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

try:
  interface = Interface() # if .MGP-config was created
except:
  interface = Interface('<user_name>','<user_password>', '<user_client_id>') # if .MGP-config was not created

#### Initialize the raster portion of the Maxar Geospatial Platform SDK

In [ ]:
raster = interface.analytics

#### Return a vsicurl formatted URL

To work with Maxar Rasters, a user will need a feature ID of a raster object. These can be found using the discovery API. If you are unfamiliar with the Discovery API, please see the Discovery and Ordering Workflow notebook. *Note: To search for raster information, utilize virtual-raster for the collections value*

For this workflow, 10300100E538FE00 will be used as the testing feature ID. Before the URL is formatted, some variables will need to be set that are needed to fully format the URL

In [ ]:
script_id = 'ortho-image'
function = "ortho"
collect_id = '10300100E538FE00'
api_token = '<your-maxar-api-token>'
crs = 'UTM'
bands = 'red,green,blue'
dra = 'true'

Pass these variables into the raster_url function

In [ ]:
raster_url = raster.raster_url(script_id=script_id, function=function, collect_id=collect_id, api_token=api_token, crs=crs, bands=bands, dra=dra)
print(raster_url)

This URL is formatted to utilize vsicurl from GDAL to interact with the raster object. By removing the vsicurl prefix from the URL, the URL can be utilized in other third party applications, such as QGIS, to display and work with rasterized data

#### Obtain raster metadata

Once a URL has been formatted, a user can visualize the metadata for the desired raster using the raster_metadata function.

In [ ]:
raster_metadata = raster.raster_metadata(raster_url)
print(raster_metadata)

#### Fetch a raster array

Utilizing the formatted URL from above, a list of raster arrays using the get_arrays function can now be fetched. A window parameter that represents the x offset in pixels, the y offset in pixels, the width in pixels, and the height in pixels of the raster object will also be declared.

In [ ]:
window = (3067, 2045, 2045, 2045)
raster_arrays = raster.get_arrays(raster_url, *window)
for array in raster_arrays:
    print(array)

#### Downloading the raster array

Once a user has their raster array created from the URL function above, the raster can be downloaded locally for further use cases. Matplotlib is required to download the raster

In [ ]:
outputpath = "<you\output\path\image.png>"
download = raster.download_raster_array(raster_arrays, outputpath)

#### Display a raster and manipulate the bands

To visualize a raster in the notebooks environment, a few modules will be needed. If these modules are not installed on your machine, be sure to do a pip install of the modules under the same python environment you are utilizing

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

A function that takes the raster information and returns an image to visualize the raster needs to be created. This function will take the raster arrays from above and utilize matplotlib for visualization

In [ ]:
def view_image(arrays):
    img = np.dstack((arrays[0], arrays[1], arrays[2]))
    plt.figure(figsize=(10,10))
    plt.imshow(img)

In [ ]:
view_image(raster_arrays)

The returned image is the raster object based off the formatted URL and the associated arrays. The bands of the raster can be changed by changing the bands variable and re-running the raster_url function and the get_arrays function on this new URL, then re-running the view_image function

In [ ]:
bands = 'nir1,red,green,blue'
raster_url = raster.raster_url(script_id=script_id, function=function, collect_id=collect_id, api_token=api_token, crs=crs, bands=bands, dra=dra)
raster_arrays = raster.get_arrays(raster_url, *window)
view_image(raster_arrays)

This image has an additional band of nir1, or Near Infrared

#### Display a tileset of a raster

To display a sliding window (tileset) of the raster, a few more functions will need to be created to make the image viewable in Jupyter

In [ ]:
def display_interactive(image_array):
    plt.close('all')
    plt.style.use('_mpl-gallery-nogrid')
    fig1, ax1 = plt.subplots()
    displayable = np.transpose(image_array, [1, 2, 0])
    im = ax1.imshow(displayable)
    plt.show()

In [ ]:
def sliding_window(url, starting_offset, window_size, step_size):
    window_width = window_size[0]
    window_height = window_size[1]

    step_x = step_size[0]
    step_y = step_size[1]

    start_x = starting_offset[0]
    start_y = starting_offset[1]

    info = gdal.Info(url, format="json")
    print('size: {}'.format(info['size']))
    image_width = info['size'][0]
    image_height = info['size'][1]

    if start_x >= image_width:
        raise RuntimeError('{} is beyond image width: {}'.format(start_x, image_width))
    if start_y >= image_height:
        raise RuntimeError('{} is beyond image height: {}'.format(start_x, image_height))

    dataset = gdal.Open(url)

    for y in range(0, image_height, step_y):
        for x in range(0, image_width, step_x):
            yield x, y, dataset.ReadAsArray(x, y, window_width, window_height)

In [ ]:
# This is the main method that walks over the image using a sliding window
def walk_the_earth(url, offset, window, step, display=False):
    for (x, y, image_array) in sliding_window(url, step_size=step, window_size=window, starting_offset=offset):
        if type(image_array) != type(None):
            if image_array.shape[1] != window[1] or image_array.shape[2] != window[0]:
                continue
            print('.', end='')
            if display:
                display_interactive(image_array)

The bands will now be reset back to the rgb colors and the raster_url and get_arrays functions will need to be re-run

In [ ]:
bands = 'red,green,blue'
raster_url = raster.raster_url(script_id=script_id, function=function, collect_id=collect_id, api_token=api_token, crs=crs, bands=bands, dra=dra)
raster_arrays = raster.get_arrays(raster_url, *window)

Some variables will be set that are needed to parse over the raster array to have the array visualized in a tileset

In [ ]:
ww = 2048
wh = 2048

sx = 1024
sy = 1024

off_x = 0
off_y = 0

The function can now be called, and a tileset will appear for the raster. Please note that this may take some time to fully finish running

In [ ]:
print('window [{}, {}]'.format(ww, wh))
print('window [{}, {}]'.format(sx, sy))
print('offset [{}, {}]'.format(0, 0))

walk_the_earth(raster_url, [off_x, off_y], [ww, wh], [sx, sy], display=True)

The bands can be manipulated in the same way as above by redefining the bands variable and running the raster_url and get_arrays functions again, and then passing those new variables into the walk_the_earth function